In [ ]:
# =============================================================================
# Code Authored by: Mehrdatapythonist
# # File: vm_allocation_optimization.py
# Version: 1.0
# Created on: 2023-05-09
# Last Updated: 2023-05-09
# Description: Implementation of Genetic Algorithm and Cuckoo Search Algorithm
#              for optimizing Virtual Machine (VM) allocation in a cloud
#              computing environment, minimizing power consumption while
#              maximizing reliability.
# Contact: https://github.com/MehrDataPythonist
# License:MIT License
# =============================================================================
import random
import math
import numpy as np

class PM:
    def __init__(self, mips, landa, pmax):
        self.mips = mips
        self.landa = landa
        self.vmlist = []
        self.pmax = pmax
        self.pmin = pmax * 0.7

class VM:
    def __init__(self, mips, l):
        self.mips = mips
        self.l = l

def calculate_energy_consumption(pm):
    if len(pm.vmlist) == 0:
        return pm.pmax * 0.7
    else:
        u_cpu = sum(vm.mips for vm in pm.vmlist) / pm.mips
        return 0.7 * pm.pmax + (0.3 * pm.pmax * u_cpu)

def calculate_vm_reliability(vm, pm):
    lambda_vm = pm.landa * (vm.mips * 100 / pm.mips) * vm.l
    return math.exp(-lambda_vm)

def calculate_pm_reliability(pm):
    return np.prod([calculate_vm_reliability(vm, pm) for vm in pm.vmlist])

def calculate_fitness(solution, pms, vms):
    psys = sum(calculate_energy_consumption(pms[i]) for i in range(len(pms)))
    rdc = np.prod([calculate_pm_reliability(pms[i]) for i in range(len(pms))])
    return psys, rdc

def generate_initial_population(pop_size, num_pms, num_vms):
    population = []
    for _ in range(pop_size):
        solution = [random.randint(0, num_pms - 1) for _ in range(num_vms)]
        population.append(solution)
    return population

def selection(population, fitnesses, num_parents):
    parents = []
    for _ in range(num_parents):
        candidates = random.sample(range(len(population)), tournament_size)
        best_candidate = min(candidates, key=lambda i: fitnesses[i][0])
        parents.append(population[best_candidate])
    return parents

def crossover(parents, offspring_size):
    offspring = []
    for _ in range(offspring_size):
        parent1, parent2 = random.sample(parents, 2)
        crossover_point = random.randint(1, len(parent1) - 1)
        offspring1 = parent1[:crossover_point] + parent2[crossover_point:]
        offspring2 = parent2[:crossover_point] + parent1[crossover_point:]
        offspring.append(offspring1)
        offspring.append(offspring2)
    return offspring

def mutation(offspring, mutation_rate):
    for i in range(len(offspring)):
        if random.random() < mutation_rate:
            mutation_point = random.randint(0, len(offspring[i]) - 1)
            offspring[i][mutation_point] = random.randint(0, num_pms - 1)
    return offspring

def levy_flight(dimension):
    beta = 1.5
    sigma = (math.gamma(1 + beta) * math.sin(math.pi * beta / 2) /
             (math.gamma((1 + beta) / 2) * beta * 2**((beta - 1) / 2)))**(1 / beta)
    u = np.random.normal(0, sigma, dimension)
    v = np.random.normal(0, 1, dimension)
    step = u / np.abs(v)**(1 / beta)
    return step

def cuckoo_search(population, fitnesses, num_cuckoos, pa):
    cuckoos = []
    for _ in range(num_cuckoos):
        cuckoo = random.choice(population)
        step_size = levy_flight(len(cuckoo))
        new_cuckoo = np.clip(cuckoo + step_size, 0, num_pms - 1).astype(int)
        cuckoos.append(new_cuckoo)

    for i in range(len(cuckoos)):
        j = random.randint(0, len(population) - 1)
        if fitnesses[j][0] > calculate_fitness(cuckoos[i], pms, vms)[0]:
            population[j] = cuckoos[i]
            fitnesses[j] = calculate_fitness(cuckoos[i], pms, vms)

    for i in range(len(population)):
        if random.random() < pa:
            population[i] = generate_initial_population(1, num_pms, num_vms)[0]
            fitnesses[i] = calculate_fitness(population[i], pms, vms)

    return population, fitnesses

def pareto_front(fitnesses):
    pareto = []
    for i in range(len(fitnesses)):
        is_dominated = False
        for j in range(len(fitnesses)):
            if i != j and fitnesses[j][0] <= fitnesses[i][0] and fitnesses[j][1] >= fitnesses[i][1]:
                is_dominated = True
                break
        if not is_dominated:
            pareto.append(i)
    return pareto

# Initialize the data center
num_pms = 10
num_vms = 50
pms = [PM(random.randint(1000, 2000), random.uniform(0.001, 0.01), 1500) for _ in range(num_pms)]
vms = [VM(random.randint(100, 500), random.uniform(0.1, 0.9)) for _ in range(num_vms)]

# Genetic Algorithm parameters
pop_size = 100
num_generations = 100
tournament_size = 5
num_parents = 20
offspring_size = 80
mutation_rate = 0.1

# Cuckoo Search parameters
num_cuckoos = 20
pa = 0.25

# Initialize the population
population = generate_initial_population(pop_size, num_pms, num_vms)
fitnesses = [calculate_fitness(solution, pms, vms) for solution in population]

# Run the hybrid algorithm
for generation in range(num_generations):
    # Genetic Algorithm
    parents = selection(population, fitnesses, num_parents)
    offspring = crossover(parents, offspring_size)
    offspring = mutation(offspring, mutation_rate)
    population.extend(offspring)
    fitnesses.extend([calculate_fitness(solution, pms, vms) for solution in offspring])

    # Cuckoo Search
    population, fitnesses = cuckoo_search(population, fitnesses, num_cuckoos, pa)

    # Update Pareto front
    pareto = pareto_front(fitnesses)

# Output the optimal solutions
print("Optimal Solutions:")
for i in pareto:
    print("Solution:", population[i])
    print("Energy Consumption (Psys):", fitnesses[i][0])
    print("Reliability (Rdc):", fitnesses[i][1])
    print()

Optimal Solutions:


In [ ]:
import random
import math
import numpy as np

class PM:
    def __init__(self, cpu, landa, pmax):
        self.cpu = cpu
        self.landa = landa
        self.pmax = pmax
        self.pmin = 0.7 * pmax
        self.vms = []

    def add_vm(self, vm):
        self.vms.append(vm)

    def remove_vm(self, vm):
        self.vms.remove(vm)

    def get_utilization(self):
        return sum(vm.mips for vm in self.vms) / self.cpu

    def get_power_consumption(self):
        u = self.get_utilization()
        if not self.vms:
            return self.pmax
        return 0.7 * self.pmax + 0.3 * self.pmax * u

class VM:
    def __init__(self, mips, l):
        self.mips = mips
        self.l = l

def calculate_reliability(pm):
    reliability = 1.0
    for vm in pm.vms:
        lvm = pm.landa * (vm.mips / pm.cpu) * vm.l
        reliability *= math.exp(-lvm)
    return reliability

def evaluate_solution(solution, pms, vms):
    for i, vm_index in enumerate(solution):
        pm = pms[vm_index]
        vm = vms[i]
        pm.add_vm(vm)

    power_consumption = sum(pm.get_power_consumption() for pm in pms)
    reliability = math.prod(calculate_reliability(pm) for pm in pms)

    for i, vm_index in enumerate(solution):
        pm = pms[vm_index]
        vm = vms[i]
        pm.remove_vm(vm)

    return power_consumption, reliability

def genetic_algorithm(pms, vms, population_size, max_generations, mutation_rate):
    def create_individual():
        return [random.randint(0, len(pms) - 1) for _ in range(len(vms))]

    def crossover(parent1, parent2):
        crossover_point = random.randint(1, len(vms) - 1)
        child1 = parent1[:crossover_point] + parent2[crossover_point:]
        child2 = parent2[:crossover_point] + parent1[crossover_point:]
        return child1, child2

    def mutate(individual):
        for i in range(len(individual)):
            if random.random() < mutation_rate:
                individual[i] = random.randint(0, len(pms) - 1)
        return individual

    population = [create_individual() for _ in range(population_size)]
    best_solution = None
    best_fitness = float('inf'), float('-inf')

    for _ in range(max_generations):
        fitness_values = [evaluate_solution(individual, pms, vms) for individual in population]
        sorted_population = [x for _, x in sorted(zip(fitness_values, population), key=lambda x: x[0])]

        best_individual = sorted_population[0]
        best_individual_fitness = fitness_values[population.index(best_individual)]
        if best_individual_fitness[0] < best_fitness[0] or (
                best_individual_fitness[0] == best_fitness[0] and best_individual_fitness[1] > best_fitness[1]):
            best_solution = best_individual
            best_fitness = best_individual_fitness

        parents = sorted_population[:population_size // 2]
        offspring = []

        while len(offspring) < population_size:
            parent1, parent2 = random.sample(parents, 2)
            child1, child2 = crossover(parent1, parent2)
            child1 = mutate(child1)
            child2 = mutate(child2)
            offspring.append(child1)
            offspring.append(child2)

        population = offspring

    return best_solution, best_fitness

def cuckoo_search(pms, vms, population_size, max_iterations, pa):
    def create_individual():
        return [random.randint(0, len(pms) - 1) for _ in range(len(vms))]

    def levy_flight(individual):
        beta = 1.5
        sigma = (math.gamma(1 + beta) * math.sin(math.pi * beta / 2) /
                 (math.gamma((1 + beta) / 2) * beta * 2 ** ((beta - 1) / 2))) ** (1 / beta)
        u = np.random.normal(0, sigma, size=len(individual))
        v = np.random.normal(0, 1, size=len(individual))
        step = u / np.abs(v) ** (1 / beta)
        individual = np.clip(individual + step, 0, len(pms) - 1).astype(int)
        return list(individual)

    nests = [create_individual() for _ in range(population_size)]
    best_solution = None
    best_fitness = float('inf'), float('-inf')

    for _ in range(max_iterations):
        for i in range(len(nests)):
            new_solution = levy_flight(nests[i])
            new_fitness = evaluate_solution(new_solution, pms, vms)
            if new_fitness[0] < best_fitness[0] or (
                    new_fitness[0] == best_fitness[0] and new_fitness[1] > best_fitness[1]):
                nests[i] = new_solution
                best_solution = new_solution
                best_fitness = new_fitness

        abandoned_nests = [nest for nest in nests if random.random() < pa]
        for nest in abandoned_nests:
            new_solution = create_individual()
            nests[nests.index(nest)] = new_solution

    return best_solution, best_fitness

# Example usage
num_pms = 5
num_vms = 10
pms = [PM(random.randint(1000, 2000), random.uniform(0.001, 0.01), random.randint(100, 200)) for _ in range(num_pms)]
vms = [VM(random.randint(100, 500), random.uniform(0.1, 0.9)) for _ in range(num_vms)]

genetic_solution, genetic_fitness = genetic_algorithm(pms, vms, population_size=50, max_generations=100, mutation_rate=0.1)
print("Genetic Algorithm:")
print("Best Solution:", genetic_solution)
print("Power Consumption:", genetic_fitness[0])
print("Reliability:", genetic_fitness[1])

cuckoo_solution, cuckoo_fitness = cuckoo_search(pms, vms, population_size=50, max_iterations=100, pa=0.25)
print("\nCuckoo Search:")
print("Best Solution:", cuckoo_solution)
print("Power Consumption:", cuckoo_fitness[0])
print("Reliability:", cuckoo_fitness[1])

Genetic Algorithm:
Best Solution: [3, 3, 3, 3, 1, 3, 2, 0, 3, 4]
Power Consumption: 627.5182997905933
Reliability: 0.9961448157270492

Cuckoo Search:
Best Solution: [3, 3, 3, 1, 3, 3, 3, 2, 0, 4]
Power Consumption: 631.8101155173005
Reliability: 0.9962991096451737


In [ ]:
import random
import math
import numpy as np

class PM:
    """
    Class representing a Physical Machine (PM).
    """
    def __init__(self, cpu, landa, pmax):
        """
        Initialize a PM object.
        :param cpu: CPU capacity of the PM.
        :param landa: Failure rate of the PM.
        :param pmax: Maximum power consumption of the PM.
        """
        self.cpu = cpu
        self.landa = landa
        self.pmax = pmax
        self.pmin = 0.7 * pmax
        self.vms = []

    def add_vm(self, vm):
        """
        Add a Virtual Machine (VM) to the PM.
        :param vm: VM object to be added.
        """
        self.vms.append(vm)

    def remove_vm(self, vm):
        """
        Remove a VM from the PM.
        :param vm: VM object to be removed.
        """
        self.vms.remove(vm)

    def get_utilization(self):
        """
        Calculate the utilization of the PM based on the allocated VMs.
        :return: Utilization value.
        """
        return sum(vm.mips for vm in self.vms) / self.cpu

    def get_power_consumption(self):
        """
        Calculate the power consumption of the PM based on its utilization.
        :return: Power consumption value.
        """
        u = self.get_utilization()
        if not self.vms:
            return self.pmax
        return 0.7 * self.pmax + 0.3 * self.pmax * u

class VM:
    """
    Class representing a Virtual Machine (VM).
    """
    def __init__(self, mips, l):
        """
        Initialize a VM object.
        :param mips: MIPS (Million Instructions Per Second) requirement of the VM.
        :param l: Load factor of the VM.
        """
        self.mips = mips
        self.l = l

def calculate_reliability(pm):
    """
    Calculate the reliability of a PM based on the allocated VMs.
    :param pm: PM object.
    :return: Reliability value.
    """
    reliability = 1.0
    for vm in pm.vms:
        lvm = pm.landa * (vm.mips / pm.cpu) * vm.l
        reliability *= math.exp(-lvm)
    return reliability

def evaluate_solution(solution, pms, vms):
    """
    Evaluate a solution by calculating the power consumption and reliability.
    :param solution: List representing the allocation of VMs to PMs.
    :param pms: List of PM objects.
    :param vms: List of VM objects.
    :return: Tuple containing power consumption and reliability.
    """
    for i, vm_index in enumerate(solution):
        pm = pms[vm_index]
        vm = vms[i]
        pm.add_vm(vm)

    power_consumption = sum(pm.get_power_consumption() for pm in pms)
    reliability = math.prod(calculate_reliability(pm) for pm in pms)

    for i, vm_index in enumerate(solution):
        pm = pms[vm_index]
        vm = vms[i]
        pm.remove_vm(vm)

    return power_consumption, reliability

def genetic_algorithm(pms, vms, population_size, max_generations, mutation_rate):
    """
    Genetic Algorithm for optimizing VM allocation.
    :param pms: List of PM objects.
    :param vms: List of VM objects.
    :param population_size: Size of the population.
    :param max_generations: Maximum number of generations.
    :param mutation_rate: Mutation rate.
    :return: Tuple containing the best solution and its fitness values.
    """
    def create_individual():
        """
        Create an individual solution.
        :return: List representing the allocation of VMs to PMs.
        """
        return [random.randint(0, len(pms) - 1) for _ in range(len(vms))]

    def crossover(parent1, parent2):
        """
        Perform crossover between two parent solutions.
        :param parent1: First parent solution.
        :param parent2: Second parent solution.
        :return: Tuple containing two offspring solutions.
        """
        crossover_point = random.randint(1, len(vms) - 1)
        child1 = parent1[:crossover_point] + parent2[crossover_point:]
        child2 = parent2[:crossover_point] + parent1[crossover_point:]
        return child1, child2

    def mutate(individual):
        """
        Perform mutation on an individual solution.
        :param individual: Individual solution to be mutated.
        :return: Mutated individual solution.
        """
        for i in range(len(individual)):
            if random.random() < mutation_rate:
                individual[i] = random.randint(0, len(pms) - 1)
        return individual

    population = [create_individual() for _ in range(population_size)]
    best_solution = None
    best_fitness = float('inf'), float('-inf')

    for _ in range(max_generations):
        fitness_values = [evaluate_solution(individual, pms, vms) for individual in population]
        sorted_population = [x for _, x in sorted(zip(fitness_values, population), key=lambda x: x[0])]

        best_individual = sorted_population[0]
        best_individual_fitness = fitness_values[population.index(best_individual)]
        if best_individual_fitness[0] < best_fitness[0] or (
                best_individual_fitness[0] == best_fitness[0] and best_individual_fitness[1] > best_fitness[1]):
            best_solution = best_individual
            best_fitness = best_individual_fitness

        parents = sorted_population[:population_size // 2]
        offspring = []

        while len(offspring) < population_size:
            parent1, parent2 = random.sample(parents, 2)
            child1, child2 = crossover(parent1, parent2)
            child1 = mutate(child1)
            child2 = mutate(child2)
            offspring.append(child1)
            offspring.append(child2)

        population = offspring

    return best_solution, best_fitness

def cuckoo_search(pms, vms, population_size, max_iterations, pa):
    """
    Cuckoo Search Algorithm for optimizing VM allocation.
    :param pms: List of PM objects.
    :param vms: List of VM objects.
    :param population_size: Size of the population.
    :param max_iterations: Maximum number of iterations.
    :param pa: Probability of abandoning a nest.
    :return: Tuple containing the best solution and its fitness values.
    """
    def create_individual():
        """
        Create an individual solution.
        :return: List representing the allocation of VMs to PMs.
        """
        return [random.randint(0, len(pms) - 1) for _ in range(len(vms))]

    def levy_flight(individual):
        """
        Perform Levy flight to generate a new solution.
        :param individual: Current solution.
        :return: New solution generated by Levy flight.
        """
        beta = 1.5
        sigma = (math.gamma(1 + beta) * math.sin(math.pi * beta / 2) /
                 (math.gamma((1 + beta) / 2) * beta * 2 ** ((beta - 1) / 2))) ** (1 / beta)
        u = np.random.normal(0, sigma, size=len(individual))
        v = np.random.normal(0, 1, size=len(individual))
        step = u / np.abs(v) ** (1 / beta)
        individual = np.clip(individual + step, 0, len(pms) - 1).astype(int)
        return list(individual)

    nests = [create_individual() for _ in range(population_size)]
    best_solution = None
    best_fitness = float('inf'), float('-inf')

    for _ in range(max_iterations):
        for i in range(len(nests)):
            new_solution = levy_flight(nests[i])
            new_fitness = evaluate_solution(new_solution, pms, vms)
            if new_fitness[0] < best_fitness[0] or (
                    new_fitness[0] == best_fitness[0] and new_fitness[1] > best_fitness[1]):
                nests[i] = new_solution
                best_solution = new_solution
                best_fitness = new_fitness

        abandoned_nests = [nest for nest in nests if random.random() < pa]
        for nest in abandoned_nests:
            new_solution = create_individual()
            nests[nests.index(nest)] = new_solution

    return best_solution, best_fitness

# Example usage
num_pms = 5
num_vms = 10
pms = [PM(random.randint(1000, 2000), random.uniform(0.001, 0.01), random.randint(100, 200)) for _ in range(num_pms)]
vms = [VM(random.randint(100, 500), random.uniform(0.1, 0.9)) for _ in range(num_vms)]

genetic_solution, genetic_fitness = genetic_algorithm(pms, vms, population_size=50, max_generations=100, mutation_rate=0.1)
print("Genetic Algorithm:")
print("Best Solution:", genetic_solution)
print("Power Consumption:", genetic_fitness[0])
print("Reliability:", genetic_fitness[1])

cuckoo_solution, cuckoo_fitness = cuckoo_search(pms, vms, population_size=50, max_iterations=100, pa=0.25)
print("\nCuckoo Search:")
print("Best Solution:", cuckoo_solution)
print("Power Consumption:", cuckoo_fitness[0])
print("Reliability:", cuckoo_fitness[1])


Genetic Algorithm:
Best Solution: [3, 3, 3, 4, 3, 2, 1, 3, 0, 3]
Power Consumption: 629.4836452662408
Reliability: 0.9959684673590158

Cuckoo Search:
Best Solution: [1, 3, 3, 0, 1, 2, 3, 1, 3, 4]
Power Consumption: 633.1709868457277
Reliability: 0.9957474351346634


In [ ]:
The comments provide a clear explanation of each class, function, and their respective parameters and return values. Here's a summary of the code:

- The `PM` class represents a Physical Machine (PM) with attributes such as CPU capacity, failure rate, power consumption, and allocated VMs.
- The `VM` class represents a Virtual Machine (VM) with attributes such as MIPS requirement and load factor.
- The `calculate_reliability` function calculates the reliability of a PM based on the allocated VMs.
- The `evaluate_solution` function evaluates a solution by calculating the power consumption and reliability.
- The `genetic_algorithm` function implements the Genetic Algorithm for optimizing VM allocation. It includes functions for creating individuals, performing crossover, mutation, and selection.
- The `cuckoo_search` function implements the Cuckoo Search Algorithm for optimizing VM allocation. It includes functions for creating individuals, performing Levy flight, and abandoning nests.
- The example usage section demonstrates how to create random PMs and VMs and run the Genetic Algorithm and Cuckoo Search Algorithm with specific parameters.

The comments provide a clear understanding of each component of the code and its purpose. They also help in maintaining and modifying the code in the future.